In [1]:
!pip install --upgrade --quiet langchain langchain-community langchain-openai chromadb streamlit langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 121.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 

In [2]:
!pip install transformers sentencepiece accelerate

In [3]:
# Import Langchain modules
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

# Other modules and packages
import os
import tempfile
import streamlit as st
import pandas as pd
from dotenv import load_dotenv

In [4]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.0/331.0 kB 30.3 MB/s eta 0:00:00


In [5]:
!pip install -U bitsandbytes>=0.46.1

In [ ]:
import os
os.environ['HF_TOKEN'] = 'HF_TOKEN_KEY_HERE'

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
import torch
# # model_id = "mistralai/Mistral-7B-Instruct-v0.1"
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# tokenizer = AutoTokenizer.from_pretrained(model_id)
# hf_model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype="auto",
#     device_map="auto",
# )

# pipe = pipeline(
#     "text-generation",
#     model=hf_model,
#     tokenizer=tokenizer,
#     max_new_tokens=1024, # Increased max_new_tokens to allow for longer JSON output
#     temperature=0.1,
#     return_full_text=False,
#     eos_token_id=tokenizer.eos_token_id
# )

# model = HuggingFacePipeline(pipeline=pipe)

import os
from openai import OpenAI

client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.environ["HF_TOKEN"],
)

def generate_llama_response(prompt: str):
    completion = client.chat.completions.create(
        model="meta-llama/Llama-3.1-70B-Instruct:scaleway",
        messages=[
            {"role": "system", "content": "You are a research assistant. Return ONLY valid JSON."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.1,
    )

    return completion.choices[0].message.content

# llm = client.chat.completions.create(
#     model="meta-llama/Meta-Llama-3-8B-Instruct",
#     messages=[
#         {"role": "system", "content": "You are a research assistant. Return ONLY valid JSON."},
#         {"role": "user", "content": "Give me the title, summary, publication date, authors of the research paper."}
#     ],
#     temperature=0.1,
#     max_tokens=1024,
#     #stream=True
# )

# def generate_llama_response(prompt: str):
#     completion = client.chat.completions.create(
#         model="meta-llama/Meta-Llama-3-8B-Instruct",
#         messages=[
#             {
#                 "role": "system",
#                 "content": [
#                     {"type": "text", "text": "You are a research assistant. Return ONLY valid JSON."}
#                 ],
#             },
#             {
#                 "role": "user",
#                 "content": [
#                     {"type": "text", "text": prompt}
#                 ],
#             },
#         ],
#         temperature=0.1,
#     )

#     return completion.choices[0].message.content

In [8]:
def generate_answer(context, question):
    prompt = f"Answer the question based only on the context.\n\nContext:\n{context}\n\nQuestion: {question}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=150)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Load PDF

In [9]:
loader = PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/explanable_ir/research.pdf")
pages = loader.load()
pages

[Document(metadata={'producer': 'pikepdf 8.15.1', 'creator': 'arXiv GenPDF (tex2pdf:57610bf)', 'creationdate': '', 'author': 'Elham Aghakhani; Rezvaneh Rezapour', 'doi': 'https://doi.org/10.48550/arXiv.2601.20747', 'license': 'http://creativecommons.org/licenses/by/4.0/', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.28 (TeX Live 2025) kpathsea version 6.4.1', 'title': 'Like a Therapist, But Not: Reddit Narratives of AI in Mental Health Contexts', 'trapped': '/False', 'arxivid': 'https://arxiv.org/abs/2601.20747v1', 'source': '/content/drive/MyDrive/Colab Notebooks/explanable_ir/research.pdf', 'total_pages': 20, 'page': 0, 'page_label': '1'}, page_content='Like a Therapist, But Not: Reddit Narratives of AI in Mental Health\nContexts\nElham Aghakhani\nDrexel University\nea664@drexel.edu\nRezvaneh Rezapour\nDrexel University\nsr3563@drexel.edu\nAbstract\nLarge language models (LLMs) are increas-\ningly used for emotional support and men-\ntal health–related interaction

## Split document

In [10]:
text_spliter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " "],
)
chunks = text_spliter.split_documents(pages)

## Create Embeddings

In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# def get_embedding():
#     embeddings = OpenAIEmbeddings(
#         model="text-embedding-ada-002", openai_api_key=OPENAI_API_KEY
#     )
#     return embeddings

def get_embedding():
    embeddings = HuggingFaceEmbeddings(
        model_name="BAAI/bge-base-en-v1.5",
        model_kwargs={"device": "cuda"}
    )
    return embeddings

embeddings = get_embedding()

/tmp/ipython-input-3701797028.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
test_vector = embeddings.embed_query("Hello world")
len(test_vector)

768

`Range:
1.0: identical meaning
0.7–0.9: very similar
0.4–0.6: moderately related
0.1–0.3: weakly related
0 or negaive: unrelated/opposite`


In [13]:
from sentence_transformers import util, SentenceTransformer

model_emb = SentenceTransformer("BAAI/bge-base-en-v1.5").to("cuda")

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: BAAI/bge-base-en-v1.5
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [14]:
prediction = "Nigeria"
reference = "nigerian"

emb1 = model_emb.encode(prediction, convert_to_tensor=True)
emb2 = model_emb.encode(reference, convert_to_tensor=True)

similarity = util.cos_sim(emb1, emb2)

print("Cosine similarity:", similarity.item())

Cosine similarity: 0.9213489294052124


## Create vector database

In [15]:
import uuid

def create_vector_db(chunks, embedding, vectorstore_path):
    # Create a list of unique ids for each document based on the content
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]

    # Ensure that only unique docs with unique ids are kept
    unique_ids = set()
    unique_chunks = []

    unique_chunks = []
    for chunk, id in zip(chunks, ids):
        if id not in unique_ids:
            unique_ids.add(id)
            unique_chunks.append(chunk)

    # Create a new Chroma database from the documents
    vectorstore = Chroma.from_documents(documents=unique_chunks,
                                        ids=list(unique_ids),
                                        embedding=embeddings,
                                        persist_directory = vectorstore_path
                                        )

    vectorstore.persist()

    return vectorstore

In [16]:
vectorstore = create_vector_db(
    chunks=chunks,
    embedding=embeddings,
    vectorstore_path="vectorstore_test"
)

/tmp/ipython-input-2529261672.py:24: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


# Get Revlevant Data

In [17]:
# Load vectorstore
vectorstore = Chroma(persist_directory="vectorstore_test", embedding_function=embeddings)

/tmp/ipython-input-1896923688.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="vectorstore_test", embedding_function=embeddings)


In [18]:
# Create retriever and get relevant chunks
retriever = vectorstore.as_retriever(search_type="similarity")
relevant_chunks = retriever.invoke("What is the title of the paper")
relevant_chunks

[Document(metadata={'license': 'http://creativecommons.org/licenses/by/4.0/', 'source': '/content/drive/MyDrive/Colab Notebooks/explanable_ir/research.pdf', 'creationdate': '', 'producer': 'pikepdf 8.15.1', 'author': 'Elham Aghakhani; Rezvaneh Rezapour', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.28 (TeX Live 2025) kpathsea version 6.4.1', 'page': 1, 'total_pages': 20, 'trapped': '/False', 'arxivid': 'https://arxiv.org/abs/2601.20747v1', 'creator': 'arXiv GenPDF (tex2pdf:57610bf)', 'doi': 'https://doi.org/10.48550/arXiv.2601.20747', 'title': 'Like a Therapist, But Not: Reddit Narratives of AI in Mental Health Contexts', 'page_label': '2'}, page_content='periences, seek advice, and access peer support\n(Thomson et al., 2024; Naslund et al., 2020). Their\naccessibility, anonymity, and asynchronous nature\nmake them especially valuable for those facing\nbarriers to traditional care, such as cost, geogra-\nphy, or stigma (Naslund et al., 2016; Merchant\net al., 2022; 

In [19]:
# PROMPT_TEMPLATE
PROMPT_TEMPLATE = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.

{context}

---

Answer the question based on the above context: {question}
"""

# Generate Response

In [20]:
from langchain_core.prompts import PromptTemplate

# Concatenate context text
context_text = "\n\n---\n\n".join([doc.page_content for doc in relevant_chunks])

# Create prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
# prompt_template = PromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(
    context=context_text,
    question="What is the title of the paper?"
)
print(prompt)

Human: 
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.

periences, seek advice, and access peer support
(Thomson et al., 2024; Naslund et al., 2020). Their
accessibility, anonymity, and asynchronous nature
make them especially valuable for those facing
barriers to traditional care, such as cost, geogra-
phy, or stigma (Naslund et al., 2016; Merchant
et al., 2022; Andalibi et al., 2017; Bouzoubaa et al.,
2024a). Reddit, in particular, has emerged as a
valuable source due to its topic-specific subreddits
(Marshall et al., 2024; Bouzoubaa et al., 2024b),
pseudonymity (Naslund et al., 2020), and commu-
nity norms that encourage candid disclosure (Ray-
land and Andrews, 2023). While this work demon-
strates the richness of online mental health dis-
course, it has largely focused on condition classifi-
cation (Dinu and Moldovan, 2021), crisi

## Using Langchain Expression Language

In [21]:
from langchain_core.runnables import RunnableLambda

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

llm_step = RunnableLambda(lambda x: generate_llama_response(x.to_string()))

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm_step
)

In [22]:
rag_chain.invoke("What's the author name?")

'{"answer": "I don\'t know"}'

## Generate Structure Responses

In [23]:
class AnswerWithSources(BaseModel):
    """An answer to the question, with sources and reasoning."""
    answer: str = Field(description="Answer to question")
    sources: str = Field(description="Full direct text chunk from the context used to answer the question")
    reasoning: str = Field(description="Explain the reasoning of the answer based on the sources")

class ExtractedInfo(BaseModel):
    """Extracted information about the research article"""
    paper_title: AnswerWithSources
    paper_summary: AnswerWithSources
    publication_year: AnswerWithSources
    paper_authors: AnswerWithSources

In [24]:
from langchain_core.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=ExtractedInfo)

In [25]:
PROMPT_TEMPLATE = """
You are a research assistant.

Use ONLY the provided context to answer.

{context}

Question:
{question}

Return your answer strictly in the following JSON format:

{{
  "paper_title": {{
    "answer": "...",
    "sources": "...",
    "reasoning": "..."
  }},
  "paper_summary": {{
    "answer": "...",
    "sources": "...",
    "reasoning": "..."
  }},
  "publication_year": {{
    "answer": "...",
    "sources": "...",
    "reasoning": "..."
  }},
  "paper_authors": {{
    "answer": "...",
    "sources": "...",
    "reasoning": "..."
  }}
}}

Return ONLY valid JSON.
Do not include explanations.
"""

In [26]:
# rag_chain = (
#             {"context": retriever | format_docs, "question": RunnablePassthrough()}
#             | prompt_template
#             | model.with_structured_output(ExtractedInfo, strict=True)
#         )

# rag_chain.invoke("Give me the title, summary, publication date, authors of the research paper.")
# prompt_template = PromptTemplate(
#     template=PROMPT_TEMPLATE
# )

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)


In [27]:
from langchain_core.runnables import RunnableLambda

llm_step = RunnableLambda(lambda x: generate_llama_response(x.to_string()))

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm_step
    | parser
)


In [28]:
rag_chain.invoke(
    "Give me the title, summary, publication date, authors of the research paper."
)

ExtractedInfo(paper_title=AnswerWithSources(answer='Youth mental health help-seeking information needs and experiences: a thematic analysis of reddit posts', sources='Meghan Sit, Sarah A Elliott, Kelsey S Wright, Shannon D Scott, and Lisa Hartling. 2024', reasoning='Title mentioned in the text'), paper_summary=AnswerWithSources(answer='The paper analyzes Reddit posts to understand youth mental health help-seeking information needs and experiences', sources='Meghan Sit, Sarah A Elliott, Kelsey S Wright, Shannon D Scott, and Lisa Hartling. 2024', reasoning='Summary inferred from the title and context'), publication_year=AnswerWithSources(answer='2024', sources='Meghan Sit, Sarah A Elliott, Kelsey S Wright, Shannon D Scott, and Lisa Hartling. 2024', reasoning='Year mentioned in the text'), paper_authors=AnswerWithSources(answer='Meghan Sit, Sarah A Elliott, Kelsey S Wright, Shannon D Scott, and Lisa Hartling', sources='Meghan Sit, Sarah A Elliott, Kelsey S Wright, Shannon D Scott, and Lis

## Transform response into a DF

In [29]:
structured_response = rag_chain.invoke("Give me the title, summary, publication date, authors of the research paper.")
df = pd.DataFrame([structured_response.model_dump()])

# Transforming into a table with two rows: 'answer' and 'source'
answer_row = []
source_row = []
reasoning_row = []

for col in df.columns:
    answer_row.append(df[col][0]['answer'])
    source_row.append(df[col][0]['sources'])
    reasoning_row.append(df[col][0]['reasoning'])

# Create new dataframe with two rows: 'answer' and 'source'
structured_response_df = pd.DataFrame([answer_row, source_row, reasoning_row], columns=df.columns, index=['answer', 'source', 'reasoning'])
structured_response_df

,paper_title,paper_summary,publication_year,paper_authors
answer,Youth mental health help-seeking information n...,The paper analyzes Reddit posts to understand ...,2024,"Meghan Sit, Sarah A Elliott, Kelsey S Wright, ..."
source,"Meghan Sit, Sarah A Elliott, Kelsey S Wright, ...","Meghan Sit, Sarah A Elliott, Kelsey S Wright, ...","Meghan Sit, Sarah A Elliott, Kelsey S Wright, ...","Meghan Sit, Sarah A Elliott, Kelsey S Wright, ..."
reasoning,Title of the research paper,Summary of the research paper,Year of publication,Authors of the research paper


# METRIC IMPLEMENTATION

In [31]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
from math import log2

In [32]:
queries = [
    "Give me the methodology of the research paper.",
]

ground_truth = {
    queries[0]: ["/content/drive/MyDrive/Colab Notebooks/explanable_ir/research.pdf"]
}

In [33]:
def precision_at_k(retrieved, relevant, k):
    retrieved_k = retrieved[:k]
    return len(set(retrieved_k) & set(relevant)) / k

def recall_at_k(retrieved, relevant, k):
    retrieved_k = retrieved[:k]
    return len(set(retrieved_k) & set(relevant)) / len(relevant)

def f1_at_k(precision, recall):
    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)

def dcg_at_k(retrieved, relevant, k):
    score = 0.0
    for i, doc_id in enumerate(retrieved[:k]):
        if doc_id in relevant:
            score += 1 / log2(i + 2)
    return score

def ndcg_at_k(retrieved, relevant, k):
    dcg = dcg_at_k(retrieved, relevant, k)

    # Ideal DCG: all relevant documents ranked at top
    ideal_relevant_count = min(len(relevant), k)

    ideal_dcg = sum(
        1 / log2(i + 2)
        for i in range(ideal_relevant_count)
    )

    return dcg / ideal_dcg if ideal_dcg > 0 else 0

In [34]:
k = 5

query = queries[0]
relevant_docs = ground_truth[query]

retrieved_docs = list(set([
    doc.metadata["source"]
    for doc in retriever.invoke(query)
]))



precision = precision_at_k(retrieved_docs, relevant_docs, k)
recall = recall_at_k(retrieved_docs, relevant_docs, k)
f1 = f1_at_k(precision, recall)
ndcg = ndcg_at_k(retrieved_docs, relevant_docs, k)

print("Retrieved:", retrieved_docs)
print("Relevant:", relevant_docs)
print("Length relevant:", len(relevant_docs))

print(f"Precision@{k}: {precision:.4f}")
print(f"Recall@{k}: {recall:.4f}")
print(f"F1@{k}: {f1:.4f}")
print(f"nDCG@{k}: {ndcg:.4f}")

Retrieved: ['/content/drive/MyDrive/Colab Notebooks/explanable_ir/research.pdf']
Relevant: ['/content/drive/MyDrive/Colab Notebooks/explanable_ir/research.pdf']
Length relevant: 1
Precision@5: 0.2000
Recall@5: 1.0000
F1@5: 0.3333
nDCG@5: 1.0000
